### Analyze.wavelet R

Python based implementation of the anaylize.wavelet function available on the [WaveletComp R](https://cran.r-project.org/web/packages/WaveletComp/) package developed by *Angi Roesch* &lt;angi@angi-stat.com&gt; and *Harald Schmidbauer* &lt;harald@hs-stat.com&gt;&lt;/harald@hs-stat.com&gt;&lt;/angi@angi-stat.com&gt;

#### Dependencies

Implementation is based on [**NumPy**](https://www.numpy.org/) library for numerical calculations.

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd

from numpy.fft import fft, ifft
from skmisc.loess import loess

# Print all values of NumPy arrays. Debugging purposes 
np.set_printoptions(threshold=sys.maxsize)

# Print all output of a cell, not only the latest value
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

pd.option_context('display.max_rows', None, 'display.max_columns', None)

#### Input dataset

Initial time series processed based on the traffic flow of a video streaming service, Youtube.

In [ ]:
# Import time saries dataset for specific service previously exported as Numpy array
my_Data = np.load('dummy.npy')

# X-axis with a range between 0 and 2016 samples taken every 5 minutes.
x = np.array(list(range(0, my_Data.shape[0])))
# Y-axis the number of bytes transferred by this type of service (both uploaded and downloaded)
y = my_Data

# Preview the dataset 
fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(211)
plt.tight_layout()
plt.plot(x,y, 'r')

#### Parameters section

Default values for function parameters, modify them as required.

In [ ]:
# Smoothing factor, as a fraction of the number of point
# to take into account for the LOESS function. Range (0,1]
# It controls the flexibility of the LOESS regression function. 
# Large values of the span produce the smoothest functions that
# wiggle the least in response to fluctuations in the data. The 
# smaller span is, the closer the regression function will conform
# to the data. Using too small a value of the smoothing parameter 
# is not desirable, however, since the regression function will 
# eventually start to capture the random error in the data
loess_span = 0.75

# Time resolution, i.e. sampling resolution on time domain, 
# 1/dt = number of intervals per time step. 
# Default: 1.
# 24h * 60m / 5 min. sample = 288
dt = 1

# Frequency resolution, i.e. sampling resolution on frequency domain, 
# 1/dj = number of suboctaves (voices per octave). 
# Default: 1/20.
dj = 1/20

# Lower Fourier period (in time units) for wavelet decomposition. 
# Default: 2*dt.
lower_period = 2 * dt  
# Upper Fourier period (in time units) for wavelet decomposition. 
# Default: (floor of one third of time series length)*dt.
upper_period = np.floor(y.size/3)*dt 

# Compute p-values
make_pval = True
# The method of generating surrogate time series
method = "white.noise"
# Number of simulations
n_sim = 100

params = None
date_format = None 
date_tz = None
verbose = True 

### Auxiliary functions

**surrogateData**: implemented only the default value available on analyze.wavelet, *white noise*.

In [ ]:
def surrogate_data(x, method = "white.noise") :

    if method != "white.noise": 
        raise NotImplementedError
        
    x_sur = np.random.normal(size = x.shape[0]) 
  
    return x_sur

**loess_data_frame**: smooth the series applying a Local Polynomial Regression Fitting if the value of the span is different from zero based on *W. S. Cleveland, E. Grosse and W. M. Shyu (1992) Local regression models. Chapter 8 of Statistical Models in S eds J.M. Chambers and T.J. Hastie, Wadsworth &amp; Brooks/Cole.*

In [ ]:
def loess_data_frame(x, y, loess_span):
    
    my_loess_y = loess(x , y,  span = loess_span)
    my_loess_y.fit()
    y_smoothed = my_loess_y.predict(x)

    return y_smoothed

### 1. Implementation of the smoothing process

The first step is apply the smoothing over our series if requested. By default, a span of 0.75 is applied.


<span style="color: green;">**Results for the next block of code have been verified with original function**</span>

In [ ]:
if loess_span != 0 :
    y_trend = loess_data_frame(x, y, loess_span)
    y_detrended = y - y_trend.values
    y_stacked = np.stack((y_detrended, y_trend.values))
    

Let's compare the smoothing process over our series versus the original one

In [ ]:
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(311)
ax.set_xticks(range(0, 2016, 288))
plt.tight_layout()
plt.plot(x,y, 'r')

ax = fig.add_subplot(312)
plt.tight_layout()
plt.plot(x,y_trend.values, 'b')

ax = fig.add_subplot(313)
ax.set_xticks(range(0, 2016, 288))
plt.tight_layout()
plt.plot(x, y_detrended, 'b')

### 2. Application of the Wavelet transformation


The function implements a single Continuos Wavelet Transformation applying the Morlet wavelet.

In [ ]:
def waveletTransform(y, dt, dj, lowerPeriod, upperPeriod) :
    series_length = y.shape[0]
    pot2 = np.trunc(np.log2(series_length) + 0.5)
    pad_length = np.power(2, (pot2 + 1))- series_length

    # Define central angular frequency omega0 and fourier factor:
    omega0 = 6
    fourier_factor = (2 * np.pi) / omega0
  
    # Compute scales and periods 
    min_scale = lower_period / fourier_factor   
    max_scale = upper_period / fourier_factor   

    J = int(np.log2(max_scale/min_scale) / dj)  
    
    scales = min_scale * np.power(2,np.array([i*dj for i in list(range(J+1))]))        
    scales_length = scales.shape[0]            
    periods = fourier_factor * scales          
    
    N = series_length + pad_length
    omega_k_direct = np.array(list(range(1,int(np.floor(N/2)+1))))
    omega_k_direct = omega_k_direct * (2 * np.pi)/(N*dt)                    # k <= N/2
    omega_k_reversed = -omega_k_direct[-2::-1]

    omega_k = np.append( [0], omega_k_direct)
    omega_k = np.append( omega_k, omega_k_reversed)
    
    # Standardize x and pad with zeros
    y_standard = (y_stacked[0] - int(np.mean(y_stacked[0]))) / int(np.round(np.std(y_stacked[0], ddof=1)))
    ypad = np.append(y_standard, np.zeros(int(pad_length)) )

    # Compute Fast Fourier Transform of xpad
    fft_ypad = fft(ypad)

    # Compute wavelet transform of x 
    # Prepare a complex matrix which accomodates the wavelet transform
    wave = np.zeros((int(scales_length),int(N)), dtype = complex)                 
    wave = wave + 1j*wave    

    # Computation for each scale...
    # ... simultaneously for all time instances
    for ind_scale in list(range(0, scales_length)) : 
    
        my_scale = scales[ind_scale]
      
        norm_factor = np.power(np.pi,(1/4)) * np.sqrt(2 * my_scale / dt)
        expnt       = -( np.power((my_scale * omega_k - omega0), 2) / 2 ) * (omega_k > 0)
        daughter    = norm_factor * np.exp(expnt)
        daughter    = daughter * (omega_k > 0)
         
        wave[ind_scale,] = ifft(fft_ypad * daughter) 

    # Cut out the wavelet transform
    wave = wave[ :, :series_length]
    
    # Compute wavelet power
    power = np.power(np.abs(wave),2) / np.reshape(np.tile(scales, series_length), (168,2016), order='F')

    # Phase  
    phase = np.angle(wave)

    # Amplitude
    ampl  = np.abs(wave) / np.reshape(np.tile(np.sqrt(scales), series_length), (168,2016), order='F')
    
    return wave, phase, ampl, periods, scales, power, series_length, scales_length


wave, phase, ampl, period, scales, power, nc, nr = waveletTransform(y_detrended, dt = dt, dj = dj, 
                        lowerPeriod = lower_period, upperPeriod = upper_period)

### 3. Computing the cone of influence COI

In [ ]:
from collections import namedtuple

COI_t = namedtuple("COI", ["x", "y", "axis_1", "axis_2"])
start = 1

def COI(start, dt, nc, nr, Period) :

    axis_1 = list(range(start, nc+1, dt))
    axis_2 = np.log2(Period)
  
    # Define central angular frequency omega0 and fourier factor:
    omega0 = 5.0  
    fourier_factor = (2 * np.pi) / omega0

    reverse_list = list(range(1, int((nc+1)/2-1)))
    reverse_list.reverse()

    coi = fourier_factor * np.sqrt(2) * dt * np.r_[1E-5, 1:int((nc+1)/2-1), reverse_list, 1E-5 ]
    coi_x = np.r_[axis_1[0]-dt*0.5, axis_1[0]-dt*0.5, axis_1, axis_1[nc-1]+dt*0.5,axis_1[nc-1]+dt*0.5  ]

    logyint = axis_2[2] - axis_2[1]
    yl = np.r_[np.log2(Period[nr-1]) + 0.5 * logyint, np.log2(Period[0]) - 0.5 * logyint]
    yr = np.flipud(yl)
    coi_y = np.r_[yl, np.log2(coi), yr]
    
    return COI_t(x = coi_x, y = coi_y, axis_1 = axis_1, axis_2 = axis_2 )

coi_x, coi_y, axis_1, axis_2 = COI(start = start, dt = dt, nc = nc, nr = nr, Period = period)

### 4. Computing the power ridge

In [ ]:
def ridge_column(column_vec, band):
    nrows = len(column_vec)
    ind = np.array(list(range(1, nrows+1)))
    band_max_vec = column_vec

    for i in range(1, band+1) :

        lower_ind = ind - i
        lower_ind[lower_ind < 1] = 1
        upper_ind = ind + i
        upper_ind[upper_ind > nrows] = nrows
        band_max_vec = np.maximum(np.maximum(band_max_vec, column_vec[lower_ind-1]), column_vec[upper_ind-1])

    my_ridge_column = np.zeros(nrows)
    my_ridge_column[np.maximum(band_max_vec, my_ridge_column) == column_vec] = 1
    return my_ridge_column

def Ridge(wavelet_spectrum, band = 5, scale_factor = 0.1):
    min_level = scale_factor * np.max(wavelet_spectrum)
    
    Ridge_in = np.apply_along_axis(ridge_column, 0, wavelet_spectrum, band = band)

    Ridge_in = Ridge_in * (wavelet_spectrum > min_level)

    return Ridge_in

rid = Ridge(power, scale_factor = 0.05)

### 5. Generating the output


In [ ]:
data = power

fig = plt.figure()
ax = fig.add_subplot(111)

x, y = np.meshgrid(
    axis_1,
    axis_2)

ax.pcolormesh(x,y, data, cmap=plt.get_cmap("Greys"))
ax.set_xticks(range(0, 2016, 288))
CS = ax.contour(x,y, rid, colors=['red'], linewidths = 1.5 )

plt.show()

In [ ]:
data = power

fig = plt.figure(figsize=(13,4))
ax = fig.add_subplot(131)

x, y = np.meshgrid(
    axis_1,
    axis_2)

ax.pcolormesh(x,y, power, cmap=plt.get_cmap("viridis"))
ax.set_yticklabels(np.power(2,np.unique(np.trunc(axis_2))))


ax = fig.add_subplot(132)
ax.pcolormesh(x,y, ampl, cmap=plt.get_cmap("viridis"))
ax.set_yticklabels(np.power(2,np.unique(np.trunc(axis_2))))

ax = fig.add_subplot(133)
ax.pcolormesh(x,y, phase, cmap=plt.get_cmap("viridis"))
ax.set_yticklabels(np.power(2,np.unique(np.trunc(axis_2))))

plt.tight_layout()
plt.show()
